In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_rows = 120
import matplotlib as mpl
import sys
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

<Figure size 640x480 with 0 Axes>

In [2]:
dataset_material = pd.read_excel('Data_Ghiringhelli.xlsx', 'Material Data')
dataset_atomic = pd.read_excel('Data_Ghiringhelli.xlsx',   'Atomic Data')
dataset_atomic.columns = ['Z', 'A', 'IP', 'EA', 'HOMO', 'LUMO', 'rs', 'rp', 'rd', '1D']
dataset_features = pd.read_csv('Feature_data.csv')
dataset_features = dataset_features.round(6)

In [3]:
dataset_atomic_array = pd.DataFrame(dataset_atomic).to_numpy()
prop_A, prop_B = [],[]
for index, row in dataset_material.iterrows():
    A = np.where(dataset_atomic_array[:,1]==dataset_material['A'][index])
    B = np.where(dataset_atomic_array[:,1]==dataset_material['B'][index])

    prop_A.append(dataset_atomic_array[A[0][0]][2:])
    prop_B.append(dataset_atomic_array[B[0][0]][2:])        

all_data = dataset_material.copy()
for index,keys in enumerate(dataset_atomic.keys()[2:-1]): #because first two para is A and B which we took care in previous cell
    all_data.insert(4, 'B_{}'.format(keys), [i[index] for i in prop_B], True)
    all_data.insert(4, 'A_{}'.format(keys), [l[index] for l in prop_A], True)

temp = []
for ii in list(all_data['Classification'].values):
    if ii == 'RS1' or ii == 'RS2' or ii == 'RS3':
        temp.append('RS')
    elif ii == 'ZB1' or ii == 'ZB2' or ii == 'ZB3':
        temp.append('ZB')
    elif ii=='N':
        temp.append('N')        

all_data['Classification'] = temp

In [4]:
import scipy
def feature_check_lr(feature_list_indexes, dataset_features, y_array):

    minvalue = float("inf")
    bestformula = ""
    fd = dict()

    fd['formulas'] = []
    fd['index']    = []
    fd['rmse']     = []
    fd['percoeff'] = []
    fd['pval']     = []
    
    dataset_keys = dataset_features.keys()[feature_list_indexes]
    for jj,keys in enumerate(dataset_keys):
            X = dataset_features[keys]
            val1, val2 = scipy.stats.pearsonr(dataset_features[keys].values, y_array.values)
            
            mse = []
            for ii in range(1000):
                X_train, X_test, y_train, y_test = train_test_split(X, y_array, test_size=0.1, random_state=ii)
                regressor = LinearRegression()
                regressor.fit((np.array(X_train)).reshape(-1,1), y_train)
                y_pred = regressor.predict((np.array(X_test)).reshape(-1,1))
                mse.append(mean_squared_error(y_test,y_pred))

            avg = float(np.average(mse))
            if avg < minvalue:
                minvalue = avg
                bestformula = keys
            #progress_bar(jj + 1, len(dataset_keys))
            fd['formulas'].append(keys)
            fd['index'].append(jj)
            fd['rmse'].append(avg)
            fd['percoeff'].append(val1)
            fd['pval'].append(val2)

    feature_rmse_dataframe = pd.DataFrame.from_dict(fd)
    print("")

    return feature_rmse_dataframe, minvalue, bestformula

In [6]:
feature_rmse_dataframe, minvalue, bestformula = feature_check_lr(range(10), dataset_features, dataset_material['DE'])

In [7]:
feature_rmse_dataframe[feature_rmse_dataframe['pval'] == np.min(feature_rmse_dataframe['pval'].values)]

,formulas,index,rmse,percoeff,pval
4,(EA_A + EA_B)/((EA_B**3)),4,0.178201,0.461423,0.000013


In [8]:
fd = feature_rmse_dataframe

In [9]:
fd[fd['rmse']==np.min(fd['rmse'].values)]['formulas'].values[0]

'(EA_A + EA_B)/((EA_A**3))'

In [10]:
np.min(fd['rmse'].values)

0.1748856048943269

In [11]:
fd[fd['percoeff']==np.max(fd['percoeff'].values)]['formulas'].values[0]

'(EA_A + EA_B)/((EA_B**3))'

In [12]:
def feature_check_lr(feature_list_indexes, dataset_features, y_array):

    minvalue = float("inf")
    bestformula = ""
    fd = dict()

    fd['formulas'] = []
    fd['index']    = []
    fd['rmse']     = []
    fd['percoeff'] = []
    fd['pval']     = []
    
    dataset_keys = dataset_features.keys()[feature_list_indexes]
    for jj,keys in enumerate(dataset_keys):
            X = dataset_features[keys]
            val1, val2 = scipy.stats.pearsonr(dataset_features[keys].values, y_array.values)
            
            mse = []
            for ii in range(1000):
                X_train, X_test, y_train, y_test = train_test_split(X, y_array, test_size=0.1, random_state=ii)
                regressor = LinearRegression()
                regressor.fit((np.array(X_train)).reshape(-1,1), y_train)
                y_pred = regressor.predict((np.array(X_test)).reshape(-1,1))
                mse.append(mean_squared_error(y_test,y_pred))

            avg = float(np.average(mse))
            
            fd['formulas'].append(keys)
            fd['index'].append(jj)
            fd['rmse'].append(avg)
            fd['percoeff'].append(val1)
            fd['pval'].append(val2)
            
    feature_rmse_dataframe = pd.DataFrame.from_dict(fd)
    fd2 = feature_rmse_dataframe.copy()
    
    bestformula_lr = fd2[fd2['rmse']==np.min(fd2['rmse'].values)]['formulas'].values[0]
    minvalue_lr    = np.min(fd2['rmse'].values)
    #--------------------------------------------------------------
    pearson_min    = np.max(fd2['percoeff'].values)
    bestformula_pearson = fd2[fd2['percoeff']==np.max(fd2['percoeff'].values)]['formulas'].values[0]
    #--------------------------------------------------------------

    print("")
    
    return feature_rmse_dataframe, ['From Linear regression :', minvalue_lr, bestformula_lr, 'From Pearson coeff :', pearson_min, bestformula_pearson]


In [13]:
feature_rmse_dataframe, bestformula = feature_check_lr(range(10), dataset_features, dataset_material['DE'])